In [6]:
from gurobipy import *
import numpy as np
import csv
import os        

In [7]:
# Demand in thousands of units
demand = np.array([15, 18, 14, 20])

# Plant capacity in thousands of units
capacity = np.array([20, 22, 17, 19, 18])

# Transportation costs per thousand units
# Hard coding the numbers for demonstration.
#transportCosts = np.array([[4000, 2500, 1200, 2200],
#              [2000, 2600, 1800, 2600],
#              [3000, 3400, 2600, 3100],
#              [2500, 3000, 4100, 3700],
#              [4500, 4000, 3000, 3200]])

# Alternatively, read 2D array from file
path = 'transportationCost.csv' # change to y ====our file directory
data = np.genfromtxt(path, dtype=str, delimiter=',', encoding='utf-8-sig')
transportCosts = data.astype(np.float)
print(transportCosts)

[[4000. 2500. 1200. 2200.]
 [2000. 2600. 1800. 2600.]
 [3000. 3400. 2600. 3100.]
 [2500. 3000. 4100. 3700.]
 [4500. 4000. 3000. 3200.]]


In [15]:
# Indices for plants and markets
plants = range(len(capacity)) # Equivalently, plants = [0,1,2,3,4]
markets = range(len(demand)) # Equivalently, markets = [0,1,2,3]

# Setting up model object
m = Model()

### Setting up Decision Variables ###

# Transportation decision variables: x[i,j] captures the
# quantity to transport to market j from plant i

# Option 1: Define variables and objective at the same time
#x = m.addVars(plants, markets, obj=transportCosts, lb = 0.0)
#m.modelSense = GRB.MINIMIZE

# Option 2: Define variables, variable bounds, and objective separately
x = m.addVars(plants, markets)


m.setObjective(sum(sum(transportCosts[i,j] * x[i,j] for i in plants) for j in markets))
m.modelSense = GRB.MINIMIZE


In [21]:
x

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (3, 0): <gurobi.Var *Awaiting Model Update*>,
 (3, 1): <gurobi.Var *Awaiting Model Update*>,
 (3, 2): <gurobi.Var *Awaiting Model Update*>,
 (3, 3): <gurobi.Var *Awaiting Model Update*>,
 (4, 0): <gurobi.Var *Awaiting Model Update*>,
 (4, 1): <gurobi.Var *Awaiting Model Update*>,
 (4, 2): <gurobi.Var *Awaiting Model Update*>,
 (4, 3): <gurobi.Var *Awaiting Model Update*>}

In [9]:
for i in plants:
    for j in markets:
        m.addConstr(x[i,j] >= 0.0)

In [12]:
transportCosts.shape

(5, 4)

In [13]:
# Demand constraints
for j in markets:
    m.addConstr(sum(x[i,j] for i in plants) >= demand[j])

# Production constraints
# Note that the right-hand limit sets the production to zero if the plant
# is closed
for i in plants:
    m.addConstr(sum(x[i,j] for j in markets) <= capacity[i])

# Alternative 1, using LinExpr object  
#for i in plants:
#    lhs = LinExpr(0)
#    rhs = capacity[i]
#    for j in markets:
#        lhs = lhs + x[i,j]
#    m.addConstr(lhs <= rhs)
    
# Alternative 2,
#for i in plants:
#    expr = LinExpr(sum(x[i,j] for j in markets))
#    m.addConstr(expr <= capacity[i])
 

# Solve
m.optimize()

# Print optimal cost
print(m.objVal)

# Print optimal solution
    
for i in plants:
    for j in markets:
        print(i, j, x[i,j].x)
        
        

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 38 rows, 20 columns and 100 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5390000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.07 seconds
Optimal objective  1.539000000e+05
153900.0
0 0 0.0
0 1 0.0
0 2 14.0
0 3 6.0
1 0 15.0
1 1 0.0
1 2 0.0
1 3 7.0
2 0 0.0
2 1 0.0
2 2 0.0
2 3 7.0
3 0 0.0
3 1 18.0
3 2 0.0
3 3 0.0
4 0 0.0
4 1 0.0
4 2 0.0
4 3 0.0
